In [41]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import wget
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [4]:
# download mediapipe model bundle
wget.download("https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task")

'hand_landmarker.task'

In [6]:
# Google's pre-written function to visualise hand landmark detection image
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [7]:
# read in and view image
img = cv2.imread("bsl_c.PNG")
cv2.namedWindow('frame', cv2.WINDOW_AUTOSIZE)
cv2.imshow('frame', img)
cv2.waitKey(0)

-1

In [8]:
# Create HandLandmarker object
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

In [16]:
# Load input image
image = mp.Image.create_from_file("bsl_c.PNG")

In [10]:
# Detect hand landmarks from input image
detection_result = detector.detect(image)

c:\Users\awright\Documents\git_repos\bsl-interpreter\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [27]:
im2 = image.numpy_view()
copied_im2_array = np.copy(im2)


In [29]:
print(copied_im2_array.shape)
type(copied_im2_array)

(548, 734, 4)


numpy.ndarray

In [33]:
bgr_img = copied_im2_array[:,:,0:3]

In [34]:
# Process classification result - visualise it
annotated_image = draw_landmarks_on_image(bgr_img, detection_result)

In [38]:
cv2.imshow('frame', img)
cv2.waitKey(0)

-1

In [44]:
x = annotated_image - bgr_img
# not reading the hands in the image as hands - need to fine tune model